In [1]:
pip install transformers datasets faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
from datasets import load_dataset

# Load your dataset
with open('/teamspace/studios/this_studio/dataset/thirukkural.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Convert to Hugging Face dataset
dataset = load_dataset('json', data_files={'train': '/teamspace/studios/this_studio/dataset/thirukkural.json'})

In [2]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import torch
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Load the tokenizer and model
tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq", 
                                         index_name="exact", 
                                         passages=dataset['train']['response'])
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq")
model.to(device)

retriver.to(device)
# Prepare the retriever with your dataset
retriever.index_passages(dataset['train']['response'])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

cuda


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

Loading dataset shards:   0%|          | 0/161 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/161 [00:00<?, ?it/s]

  0%|          | 0/21016 [00:00<?, ?it/s]

In [6]:
from transformers import Trainer, TrainingArguments
import torch

# Prepare the dataset for training
def preprocess_function(examples):
    inputs = [ex['instruction'] for ex in examples]
    targets = [ex['response'] for ex in examples]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the labels
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

# Preprocess the dataset
train_dataset = dataset['train'].map(preprocess_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start training
trainer.train()

Map:   0%|          | 0/1330 [00:00<?, ? examples/s]

TypeError: string indices must be integers

In [ ]:
def answer_question(question):
    inputs = tokenizer(question, return_tensors="pt")
    generated_ids = model.generate(**inputs)
    answer = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return answer

# Example usage
question = "What is Kural number 1?"
print(answer_question(question))